<p style="font-family: helvetica,arial,sans-serif; font-size:2.0em;color:white; background-color: black">&emsp;<b>Use Case 2 High-rise Building and Public Transport</b></p>
    
<p style="font-family: helvetica,arial,sans-serif; font-size:1.6em;color:black; background-color: #DDDDDD; text-align:justify">&emsp;<b>Authored by: </b> Jack Pham, Andrew Tilling</p>

<p style="font-family: helvetica,arial,sans-serif; font-size:1.6em;color:white; background-color: black">&emsp;<b>Scenario</b>

#### {Using User Story format, write a description of the problem you are trying to solve for this use case.}

In [ ]:
!pip install pygeos

!pip install folium matplotlib mapclassify
!pip install geopandas
!pip install tqdm
import folium
import pandas as pd
import os
import time
from datetime import datetime
import numpy as np
import pyproj
import pygeos

import plotly.express as px



import geopandas as gpd
import shapely.geometry
import folium
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [10]:
df = pd.read_csv('https://data.melbourne.vic.gov.au/api/views/rm92-h5tq/rows.csv?accessType=DOWNLOAD')

Showing the map of Melbourne

In [11]:
m = folium.Map([-37.8,145],zoom_start=11)
m

Grouping buildings by block ID and CLUE small area and aggregating on Dwelling number.

In [12]:
groupbyfields = ['CLUE small area','Block ID']
aggregatebyfields = {'Dwelling number': ["sum"],
                    'y coordinate':["first"],
                     'x coordinate':["first"]}

dwellingsByLocn = pd.DataFrame(df.groupby(groupbyfields, as_index=False).agg(aggregatebyfields))
dwellingsByLocn.columns = dwellingsByLocn.columns.get_level_values(0)
dwellingsByLocn

,CLUE small area,Block ID,Dwelling number,y coordinate,x coordinate
0,Carlton,201,250,-37.79430,144.96594
1,Carlton,202,47,-37.79568,144.96601
2,Carlton,203,81,-37.79681,144.96582
3,Carlton,204,567,-37.79783,144.96517
4,Carlton,205,84,-37.79900,144.96476
...,...,...,...,...,...
445,West Melbourne (Residential),439,25,-37.80883,144.95290
446,West Melbourne (Residential),440,519,-37.80952,144.95061
447,West Melbourne (Residential),441,18,-37.80972,144.94907
448,West Melbourne (Residential),443,84,-37.81055,144.94989


Plotting data on high rise dwellings in Melbourne city. Size of bubble denotes the number of residents. 

In [13]:
fig = px.scatter_mapbox(dwellingsByLocn, lat="y coordinate", lon="x coordinate",
                        hover_name="CLUE small area",
                        hover_data=["CLUE small area", "Block ID"],
                        color_continuous_scale=px.colors.sequential.Plasma, zoom=12, height=600,
                        color="Dwelling number",
                        size = "Dwelling number"
                       )
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Extracting data for busstops in Melbourne

In [6]:
bus_stops_df = pd.read_csv("https://data.melbourne.vic.gov.au/api/views/vzsu-xnf6/rows.csv?accessType=DOWNLOAD")
bus_stops_df = bus_stops_df[['the_geom','OBJECTID']]
bus_stops_df['Longitude'] = bus_stops_df['the_geom'].apply(lambda s: s.split()[1][1:]).astype(float)
bus_stops_df['Latitude'] = bus_stops_df['the_geom'].apply(lambda s: s.split()[2][:-1]).astype(float)
bus_stops_df = bus_stops_df.drop('the_geom',axis=1)
bus_stops_df.head(3)

,OBJECTID,Longitude,Latitude
0,303,144.921856,-37.787445
1,355,144.932393,-37.803842
2,434,144.963813,-37.816247


In [7]:


rest_VIC = gpd.read_file(
    "https://data.melbourne.vic.gov.au/api/geospatial/ss79-v558?method=export&format=GeoJSON"
)

fig = px.scatter_mapbox(bus_stops_df, lat="Latitude", lon="Longitude",
                        hover_name="OBJECTID",
                        color_continuous_scale=px.colors.sequential.Plasma, zoom=12, height=600,
                       )
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})



fig.show()


NameError: ignored

In [ ]:
#Creating the dwellings geodataframe
dwellings_gdf = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df['x coordinate'], df['y coordinate'])) #.drop(['x coordinate','y coordinate'], axis=1)
#Reprojecting to planar coordinates
dwellings_gdf = dwellings_gdf.set_crs(pyproj.CRS.from_user_input('EPSG:4326'))
dwellings_gdf = dwellings_gdf.to_crs( pyproj.CRS.from_user_input('EPSG:28355'))

#Creating bus stop geodataframe
bus_stops_gdf = gpd.read_file('data/mobility/bus/Bus stops/geo_export_1bf369d0-b42b-454c-8126-f76e891e47a6.shp')
#Reprojecting to planar coordinates
bus_stops_gdf = bus_stops_gdf.to_crs( pyproj.CRS.from_user_input('EPSG:28355'))

#Creating clue_blocks geodataframe
clue_blocks_gdf =  gpd.read_file('data/limits/Blocks for Census of Land Use and Employment (CLUE) (1)/geo_export_17bc9af5-4337-4cf6-96d8-b54bd808b156.shp')
clue_blocks_gdf = clue_blocks_gdf.to_crs( pyproj.CRS.from_user_input('EPSG:28355'))

#Creating clue small areas geodataframe
clue_small_areas_gdf = gpd.read_file('data/limits/Small Areas for Census of Land Use and Employment (CLUE)/geo_export_57442dbe-773d-4952-9d62-ed5790c20acb.shp')
clue_small_areas_gdf = clue_small_areas_gdf.to_crs( pyproj.CRS.from_user_input('EPSG:28355'))


In [ ]:
def get_k_closest_geom_distance(gdf1, gdf2, K):
    '''
    Finds the mean distance between each geometry in gdf1 and the K nearest geometries of gdf2
    '''
    distances = []
    for point in tqdm(gdf1['geometry']):
        distances.append(gdf2['geometry'].distance(point).sort_values()[:K].mean())

    return distances

In [ ]:
def get_points_in_radius(gdf1, gdf2, R):
    ''' 
    finds which points from gdf2 are within R meters from each point in gdf2
    
    Returns:
        A dictionary with 2 lists. 
        One of the lists shows for each geometry (in gdf1), the number of geometries (in gdf2) that are within R meters.
        The other list shows for each geometry (in gdf1), the mean distance for the geometries (in gdf2) that are within R meters
    '''
    
    radius_geoms = gdf1['geometry'].buffer(R)
    result = {f'number_of_geoms_within_{R}m':[],f'mean_distance_to_geoms_within_{R}m':[]}
    for i, radius_geom in enumerate(tqdm(radius_geoms)):
        intersection_geoms = gdf2.intersection(radius_geom)
        valid_geoms_mask = ~intersection_geoms.is_empty
        number_of_geoms = len(intersection_geoms[valid_geoms_mask])
        if number_of_geoms>0:
            mean_distance_to_geoms = np.array([ gdf1['geometry'][i].distance(point) for point in intersection_geoms[valid_geoms_mask]]).mean()
        else:
            mean_distance_to_geoms = np.nan
            
        result[f'number_of_geoms_within_{R}m'].append(number_of_geoms)
        result[f'mean_distance_to_geoms_within_{R}m'].append(mean_distance_to_geoms)
        
    return result

In [ ]:
#Getting the distance from each property to the nearest bus stops
distances_nearest_bus_stop = get_k_closest_geom_distance(dwellings_gdf, bus_stops_gdf, K=1)
#Getting the mean distance from each property to the 5 nearest bus stops
distances_5_nearest_bus_stop = get_k_closest_geom_distance(dwellings_gdf, bus_stops_gdf, K=5)

#Finds how many bus stops are within 300 meters of each property
#and what is the mean distance between each property and the bus stops that are within 500 meters
R = 300
distance_radius_result = get_points_in_radius(dwellings_gdf, bus_stops_gdf, R=R)

#Creating bus stop distances dataframe with the results obtained previuosly
bus_stops_distances_info_df = pd.DataFrame(distance_radius_result)
bus_stops_distances_info_df.columns = [f'n_bus_stops_within_{R}m', f'mean_dist_to_bus_stops_within_{R}m']
bus_stops_distances_info_df['dist_from_nearest_1_bus_stops'] = distances_nearest_bus_stop
bus_stops_distances_info_df['dist_from_nearest_5_bus_stops'] = distances_5_nearest_bus_stop
bus_stops_distances_info_df['Property ID'] = dwellings_gdf['Property ID']
bus_stops_distances_info_df['Base property ID'] = dwellings_gdf['Base property ID']
#saving result ot csv
#bus_stops_distances_info_df.to_csv('output/bus_stop_distances.csv', index=False)

#Adding the created columns to the dwellings geoDataFrame
for column in bus_stops_distances_info_df.columns:
    if column not in dwellings_gdf.columns:
        dwellings_gdf[column] = bus_stops_distances_info_df[column] 

100%|██████████| 10402/10402 [00:32<00:00, 318.46it/s]


In [ ]:
def create_grid(gdf, n_cells=15):

    '''
    Creates a regular grid over the extent of gdf

    Returns:
        A GeoDataFrame with the cells geometries
    '''
    # total area for the grid
    xmin, ymin, xmax, ymax= gdf.total_bounds
    # how many cells across and down
    cell_size = (xmax-xmin)/n_cells
    # projection of the grid
    #crs = "+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs"
    crs = gdf.crs
    # create the cells in a loop
    grid_cells = []
    for x0 in np.arange(xmin, xmax+cell_size, cell_size ):
        for y0 in np.arange(ymin, ymax+cell_size, cell_size):
            # bounds
            x1 = x0-cell_size
            y1 = y0+cell_size
            grid_cells.append( shapely.geometry.box(x0, y0, x1, y1)  )
    grid = gpd.GeoDataFrame(grid_cells, columns=['geometry'], 
                                    crs=crs)
    return grid

In [ ]:
def summarize_within(input_gdf, input_summary_gdf, in_fields, out_fields = None, aggfunc='mean'):
    '''
    Overlays a polygon layer with another layer to calculate attribute field statistics about those features (input_summary_gdf) within the polygons (input_gdf).

    Parameters:
        input_gdf: Geodataframe of the polygons in which features will be summarized by.
        input_summary_gdf: Geodataframe of features that will be summarized
        in_fields: name of the fields (in input_summary_gdf) that will be summarized
        out_fields: name that the fields will have after they're summarized
        aggfunc: function that will be used to summarize
    
    Returns:
        A geodataframe with 'input_gdf' polygons and the attributes of 'input_summary_gdf' summarized by each polygon.

    '''
    input_gdf = input_gdf.copy()
    input_summary_gdf = input_summary_gdf.copy()
    if out_fields == None:
        out_fields = in_fields
    #Merges the dwelling points with the input_polygons. A new column "index right" is created. It indicates in what cell the property is within.
    merged = gpd.sjoin(input_summary_gdf, input_gdf, how='left')
    #Now lets count how many properties are within each cell
    dissolve = merged.dissolve(by="index_right", aggfunc=aggfunc) #Dissolve (looks like groupby) by the cell index
    for in_field, out_field in zip(in_fields, out_fields):
        input_gdf.loc[dissolve.index, out_field] = dissolve[in_field].values #Putting number of properties in input_polygons gdf

    return input_gdf.round(2)
    #input_polygons = input_polygons.dropna().round(2)

#### Summarizing by grid cell

In [ ]:
#Creating the grid GeoDataFrame
grids = create_grid(dwellings_gdf)
dwellings_gdf['n_properties']=1 #initialize n_properties to 1, so we can sum how many properties in each grid cell
#Summarizing Dwelling GeodataFrame by sum
summarized_grid = summarize_within(
    grids, 
    dwellings_gdf,
     in_fields= ['n_properties', 'Dwelling number'], 
     out_fields= ['Properties', 'Dwelling number'],
     aggfunc='sum')
#Summarizing Dwelling GeodataFrame by mean
summarized_grid = summarize_within(
    input_gdf=summarized_grid, 
    input_summary_gdf=dwellings_gdf,
    in_fields = ['n_bus_stops_within_300m','mean_dist_to_bus_stops_within_300m', 'dist_from_nearest_1_bus_stops','dist_from_nearest_5_bus_stops'], 
    out_fields= ['Average number of bus stops within 300m of each property','Average distance from each property and bus stops in a 300m radius','Average distance between properties and the nearest bus stop', 'Average distance between properties and the 5 nearest bus stops'], 
    aggfunc='mean')
summarized_grid.dropna().head(1)

#Summarizing Bus Stops GeodataFrame by sum
bus_stops_gdf['n_bus_stops'] = 1 #initialize bus stops to 1, so we can sum how many bus stops in each grid cell
summarized_grid = summarize_within(
    input_gdf=summarized_grid, 
    input_summary_gdf=bus_stops_gdf,
    in_fields = ['n_bus_stops'], 
    out_fields= ['Total bus stops'], 
    aggfunc='sum')
summarized_grid.dropna().head(1)

summarized_grid['Total bus stops'].fillna(0, inplace=True)


summarized_grid.explore(
     column="Dwelling number", # make choropleth based on "BoroName" column
     tooltip=[el for el in summarized_grid.columns if el!='geometry'], # show "BoroName" value in tooltip (on hover)
     scheme="naturalbreaks",
     popup=True, # show all values in popup (on click)
     tiles="CartoDB positron", # use "CartoDB positron" tiles
     cmap="Reds", # use "Set1" matplotlib colormap
     style_kwds=dict(color="black") # use black outline,
    )

#### Summarizing by CLUE blocks

In [ ]:
dwellings_gdf['n_properties']=1 #initialize n_properties to 1, so we can sum how many properties in each grid cell
#Summarizing Dwelling GeodataFrame by sum
summarized_clue_blocks = summarize_within(
    clue_blocks_gdf, 
    dwellings_gdf,
     in_fields= ['n_properties', 'Dwelling number'], 
     out_fields= ['Properties', 'Dwelling number'],
     aggfunc='sum')

#Summarizing Dwelling GeodataFrame by mean
summarized_clue_blocks = summarize_within(
    input_gdf=summarized_clue_blocks, 
    input_summary_gdf=dwellings_gdf,
    in_fields = ['n_bus_stops_within_300m','mean_dist_to_bus_stops_within_300m','dist_from_nearest_1_bus_stops','dist_from_nearest_5_bus_stops'], 
    out_fields= ['Average number of bus stops within 300m of each property','Average distance from each property and bus stops in a 300m radius','Average distance between properties and the nearest bus stop', 'Average distance between properties and the 5 nearest bus stops'], 
    aggfunc='mean')

#Summarizing Bus Stops GeodataFrame by sum
bus_stops_gdf['n_bus_stops'] = 1 #initialize bus stops to 1, so we can sum how many bus stops in each grid cell
summarized_clue_blocks = summarize_within(
    input_gdf=summarized_clue_blocks, 
    input_summary_gdf=bus_stops_gdf,
    in_fields = ['n_bus_stops'], 
    out_fields= ['Total bus stops'], 
    aggfunc='sum')

summarized_clue_blocks['Total bus stops'].fillna(0, inplace=True)

summarized_clue_blocks.explore(
     column="Dwelling number", # make choropleth based on "BoroName" column
     tooltip=[el for el in summarized_clue_blocks.columns if el!='geometry'], # show "BoroName" value in tooltip (on hover)
     scheme="naturalbreaks",
     popup=True, # show all values in popup (on click)
     tiles="CartoDB positron", # use "CartoDB positron" tiles
     cmap="Reds", # use "Set1" matplotlib colormap
     style_kwds=dict(color="black") # use black outline,
    )

#### Summarize by Clue Small Areas

In [ ]:
dwellings_gdf['n_properties']=1 #initialize n_properties to 1, so we can sum how many properties in each grid cell
#Summarizing Dwelling GeodataFrame by sum
summarized_clue_small_areas = summarize_within(
    clue_small_areas_gdf, 
    dwellings_gdf,
     in_fields= ['n_properties', 'Dwelling number'], 
     out_fields= ['Properties', 'Dwelling number'],
     aggfunc='sum')

#Summarizing Dwelling GeodataFrame by mean
summarized_clue_small_areas = summarize_within(
    input_gdf=summarized_clue_small_areas, 
    input_summary_gdf=dwellings_gdf,
    in_fields = ['n_bus_stops_within_300m','mean_dist_to_bus_stops_within_300m','dist_from_nearest_1_bus_stops','dist_from_nearest_5_bus_stops'], 
    out_fields= ['Average number of bus stops within 300m of each property','Average distance from each property and bus stops in a 300m radius','Average distance between properties and the nearest bus stop', 'Average distance between properties and the 5 nearest bus stops'], 
    aggfunc='mean')

#Summarizing Bus Stops GeodataFrame by sum
bus_stops_gdf['n_bus_stops'] = 1 #initialize bus stops to 1, so we can sum how many bus stops in each grid cell
summarized_clue_small_areas = summarize_within(
    input_gdf=summarized_clue_small_areas, 
    input_summary_gdf=bus_stops_gdf,
    in_fields = ['n_bus_stops'], 
    out_fields= ['Total bus stops'], 
    aggfunc='sum')

summarized_clue_small_areas['Total bus stops'].fillna(0, inplace=True)

summarized_clue_small_areas.explore(
     column="Dwelling number", # make choropleth based on "BoroName" column
     tooltip=[el for el in summarized_clue_small_areas.columns if el not in ['geometry','shape_area', 'shape_len']], # show "BoroName" value in tooltip (on hover)
     scheme="naturalbreaks",
     popup=True, # show all values in popup (on click)
     tiles="CartoDB positron", # use "CartoDB positron" tiles
     cmap="Reds", # use "Set1" matplotlib colormap
     style_kwds=dict(color="black") # use black outline,
    )

Transport data source: http://data.ptv.vic.gov.au/downloads/gtfs.zip

# Action Plan
* Get data on transportation systems in melbourne
* Plot them together with high-rise building data
* Find the closest transportation point to each high rise dwelling
* Calculate the average distance between a dwelling and a transportation point. 
* Plot these distances on the map to see how each district performs

## Next Analysis:
* Developing buildings vs transportation points

## Secondary Analysis:
* Availability of public transport vs median income(demographic data)
* High rise buildings vs median income(demographic data)

## Livability:
* Livability score vs transportation points